In [1]:
import numpy as np
import pandas as pd
import keras
import tensorflow as tf
from tensorflow.python.client import device_lib
device_lib.list_local_devices() 

Using TensorFlow backend.


[name: "/cpu:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 15583264866943820827, name: "/gpu:0"
 device_type: "GPU"
 memory_limit: 2790653952
 locality {
   bus_id: 1
 }
 incarnation: 9856235368355860618
 physical_device_desc: "device: 0, name: GeForce GTX 780, pci bus id: 0000:02:00.0"]

In [2]:
side_length = 124 # side lenght of the images

In [3]:
def get_shuffled_splitted_data(path):
    df = pd.read_csv(path)

    # Shuffle and split data
    
    X_train, X_test, X_val = np.split(df.sample(frac=1), [int(.7*len(df)), int(.9*len(df))])
    
    # Pop labels and transform them to vectors
    y_train, y_test, y_val = X_train.pop("label"), X_test.pop("label"), X_val.pop("label")
    y_train, y_test, y_val = y_train.values.reshape((-1, 1)), y_test.values.reshape((-1, 1)), y_val.values.reshape((-1, 1))
    
    # Reshape the features for CNN
    X_train = X_train.as_matrix().reshape(X_train.shape[0], 1, side_length, side_length)
    X_test = X_test.as_matrix().reshape(X_test.shape[0], 1, side_length, side_length)
    X_val = X_val.as_matrix().reshape(X_val.shape[0], 1, side_length, side_length)
    
    # Norm data
    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
    X_val = X_val.astype('float32')
    X_train /= 255
    X_test /= 255
    X_val /= 255
    
    # Convert labels to categorical values
    y_train = keras.utils.to_categorical(y_train, 2)
    y_test = keras.utils.to_categorical(y_test, 2)
    y_val = keras.utils.to_categorical(y_val, 2)
    
    return X_train, y_train, X_test, y_test, X_val, y_val
    
X_train, y_train, X_test, y_test, X_val, y_val = get_shuffled_splitted_data('data/subset-2-HnyTnz.txt.csv')

In [80]:
with tf.device('/gpu:0'):

    import keras
    from keras.preprocessing.image import ImageDataGenerator
    from keras.models import Sequential
    from keras.layers import Dense, Dropout, Activation, Flatten, Cropping2D
    from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
    from keras import backend as K
    from keras.losses import categorical_crossentropy
    from keras.optimizers import Adadelta, Nadam
    K.set_image_dim_ordering('th')

    batch_size = 32
    num_classes = 2
    epochs = 4

    # The data, shuffled and split between train and test sets:
    print('X_train shape:', X_train.shape)
    print(X_train.shape[0], 'train samples')
    print(X_test.shape[0], 'test samples')
    print(X_val.shape[0], 'validation samples')

    model = Sequential()
  
    model.add(Cropping2D(cropping=((48, 48), (0, 48)), input_shape=(1, side_length, side_length)))
  

    model.add(Conv2D(4, kernel_size=(3, 3)))
    model.add(MaxPooling2D(pool_size=(2, 4)))
   
    model.add(Conv2D(48, (1, 1)))
    
    model.add(Flatten())
    model.add(Dense(5))
    model.add(Dense(10))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(loss=categorical_crossentropy,
                  optimizer=Nadam(),
                  metrics=['accuracy'])

    model.fit(X_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data=(X_val, y_val))

    test_score = model.evaluate(X_test, y_test, verbose=0)
    print('Test loss:', test_score[0])
    print('Test accuracy:', test_score[1])
    
    val_score = model.evaluate(X_val, y_val, verbose=0)
    print('Val loss:', val_score[0])
    print('Val accuracy:', val_score[1])

X_train shape: (3497, 1, 124, 124)
3497 train samples
1000 test samples
500 validation samples
Train on 3497 samples, validate on 500 samples
Epoch 1/4
3497/3497 [==============================] - 4s - loss: 0.1227 - acc: 0.9508 - val_loss: 0.0324 - val_acc: 0.9960
Epoch 2/4
3497/3497 [==============================] - 2s - loss: 0.0340 - acc: 0.9908 - val_loss: 0.0122 - val_acc: 1.0000
Epoch 3/4
3497/3497 [==============================] - 2s - loss: 0.0084 - acc: 0.9980 - val_loss: 0.0034 - val_acc: 1.0000
Epoch 4/4
3497/3497 [==============================] - 2s - loss: 0.0030 - acc: 0.9997 - val_loss: 0.0017 - val_acc: 1.0000
Test loss: 0.000885369949043
Test accuracy: 1.0
Val loss: 0.00168039796501
Val accuracy: 1.0


In [61]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cropping2d_53 (Cropping2D)   (None, 1, 28, 76)         0         
_________________________________________________________________
conv2d_103 (Conv2D)          (None, 4, 26, 74)         40        
_________________________________________________________________
max_pooling2d_49 (MaxPooling (None, 4, 13, 18)         0         
_________________________________________________________________
conv2d_104 (Conv2D)          (None, 48, 13, 18)        240       
_________________________________________________________________
flatten_53 (Flatten)         (None, 11232)             0         
_________________________________________________________________
dense_104 (Dense)            (None, 10)                112330    
_________________________________________________________________
dense_105 (Dense)            (None, 10)                110       
__________